<a href="https://colab.research.google.com/github/romangcode38/Thesis_NN-GA/blob/main/GA%2BNN%2BMNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
import math
from typing import Dict, Tuple

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.layers import (Input, Conv2D, MaxPooling2D, Flatten,
                                     Dense, Dropout)
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam



In [2]:
# ============================================================
# 1) Load MNIST (digits)
# ============================================================

def load_mnist(val_ratio=0.1, seed=7):
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

    # normalize + add channel dim: (N, 28, 28, 1)
    x_train = (x_train.astype("float32") / 255.0)[..., None]
    x_test  = (x_test.astype("float32") / 255.0)[..., None]

    # split train into train/val
    rng = np.random.default_rng(seed)
    idx = np.arange(len(x_train))
    rng.shuffle(idx)

    split = int((1 - val_ratio) * len(x_train))
    tr_idx, va_idx = idx[:split], idx[split:]

    x_tr, y_tr = x_train[tr_idx], y_train[tr_idx]
    x_va, y_va = x_train[va_idx], y_train[va_idx]

    return x_tr, y_tr, x_va, y_va, x_test, y_test


In [3]:
# ============================================================
# 2) Build CNN model (generic digits classifier)
# ============================================================

def build_cnn(ind: Dict) -> Model:
    inp = Input(shape=(28, 28, 1), dtype="float32")

    x = Conv2D(ind["conv_filters"], kernel_size=3, activation="relu", padding="same")(inp)
    x = MaxPooling2D(pool_size=2)(x)

    x = Conv2D(ind["conv_filters"] * 2, kernel_size=3, activation="relu", padding="same")(x)
    x = MaxPooling2D(pool_size=2)(x)

    x = Flatten()(x)
    x = Dense(ind["dense_units"], activation="relu")(x)
    if ind["dropout"] > 0:
        x = Dropout(ind["dropout"])(x)

    out = Dense(10, activation="softmax")(x)

    model = Model(inp, out)
    model.compile(
        optimizer=Adam(learning_rate=ind["learning_rate"]),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model

In [4]:
# ============================================================
# 3) GA search space
# ============================================================

GA_SPACE = {
    "conv_filters": [16, 24, 32, 48, 64],
    "dense_units": [32, 64, 96, 128, 160, 192],
    "dropout": (0.0, 0.6),
    "learning_rate": (1e-4, 5e-3),
    "batch_size": [32, 64, 128],
    "epochs_fast": [1, 2, 3],  # evaluare rapidă
}

In [5]:
def make_random_individual():
    return {
        "conv_filters": random.choice(GA_SPACE["conv_filters"]),
        "dense_units": random.choice(GA_SPACE["dense_units"]),
        "dropout": random.uniform(*GA_SPACE["dropout"]),
        "learning_rate": 10 ** random.uniform(
            math.log10(GA_SPACE["learning_rate"][0]),
            math.log10(GA_SPACE["learning_rate"][1]),
        ),
        "batch_size": random.choice(GA_SPACE["batch_size"]),
        "epochs_fast": random.choice(GA_SPACE["epochs_fast"]),
    }

In [6]:
def tournament_selection(pop, fits, k=3):
    idxs = random.sample(range(len(pop)), k)
    best = max(idxs, key=lambda i: fits[i])
    return pop[best]

In [7]:
def crossover(p1, p2):
    child = {}
    child["conv_filters"] = random.choice([p1["conv_filters"], p2["conv_filters"]])
    child["dense_units"] = random.choice([p1["dense_units"], p2["dense_units"]])
    child["batch_size"] = random.choice([p1["batch_size"], p2["batch_size"]])
    child["epochs_fast"] = random.choice([p1["epochs_fast"], p2["epochs_fast"]])

    alpha = random.random()
    child["dropout"] = alpha * p1["dropout"] + (1 - alpha) * p2["dropout"]

    a = random.random()
    child["learning_rate"] = (p1["learning_rate"] ** a) * (p2["learning_rate"] ** (1 - a))
    return child

In [8]:
def mutate(ind, prob=0.25):
    if random.random() < prob:
        ind["conv_filters"] = random.choice(GA_SPACE["conv_filters"])
    if random.random() < prob:
        ind["dense_units"] = random.choice(GA_SPACE["dense_units"])
    if random.random() < prob:
        lo, hi = GA_SPACE["dropout"]
        ind["dropout"] = min(hi, max(lo, ind["dropout"] + random.uniform(-0.10, 0.10)))
    if random.random() < prob:
        lo, hi = GA_SPACE["learning_rate"]
        ind["learning_rate"] *= random.uniform(0.6, 1.6)
        ind["learning_rate"] = min(hi, max(lo, ind["learning_rate"]))
    if random.random() < prob:
        ind["batch_size"] = random.choice(GA_SPACE["batch_size"])
    if random.random() < prob:
        ind["epochs_fast"] = random.choice(GA_SPACE["epochs_fast"])
    return ind

In [9]:
# ============================================================
# 4) Fitness function: val accuracy after short training
# ============================================================

def key_ind(ind):
    return (
        ind["conv_filters"],
        ind["dense_units"],
        round(ind["dropout"], 4),
        round(ind["learning_rate"], 8),
        ind["batch_size"],
        ind["epochs_fast"],
    )

In [10]:
def ga_fitness(ind, x_tr, y_tr, x_va, y_va, cache, seed_base=1234) -> float:
    k = key_ind(ind)
    if k in cache:
        return cache[k]

    tf.keras.utils.set_random_seed(seed_base + abs(hash(k)) % (2**31 - 1))
    K.clear_session()

    model = build_cnn(ind)

    # optional: train on a subset for speed (poți scoate asta pe GPU)
    # sub = 12000
    # x_tr_s, y_tr_s = x_tr[:sub], y_tr[:sub]
    x_tr_s, y_tr_s = x_tr, y_tr

    model.fit(
        x_tr_s, y_tr_s,
        epochs=ind["epochs_fast"],
        batch_size=ind["batch_size"],
        verbose=0
    )

    _, acc = model.evaluate(x_va, y_va, verbose=0)
    cache[k] = float(acc)
    return float(acc)


In [11]:
def run_ga(
    x_tr, y_tr, x_va, y_va,
    pop_size=10,
    generations=10,
    elite=2,
    cx_rate=0.8,
    mut_rate=0.25,
    fitness_target=0.985,
    min_generations=4,
    patience=4,
    seed=7,
    print_best_each_gen=True
):
    random.seed(seed)
    cache = {}

    pop = [make_random_individual() for _ in range(pop_size)]

    best_overall = None
    best_overall_fit = -1.0
    stagnation = 0

    # history: păstrăm best_ind per generație
    history = {
        "gen": [],
        "best_fit": [],
        "avg_fit": [],
        "best_ind": [],      # dict pentru fiecare gen
        "best_overall_fit": []
    }

    for g in range(generations):
        fits = [ga_fitness(ind, x_tr, y_tr, x_va, y_va, cache) for ind in pop]

        best_fit = float(max(fits))
        avg_fit = float(sum(fits) / len(fits))
        best_idx = int(np.argmax(fits))
        best_ind = dict(pop[best_idx])  # copiem ca să nu se modifice ulterior

        # update best overall
        if best_fit > best_overall_fit:
            best_overall_fit = best_fit
            best_overall = dict(best_ind)
            stagnation = 0
        else:
            stagnation += 1

        # salvăm în history
        history["gen"].append(g + 1)
        history["best_fit"].append(best_fit)
        history["avg_fit"].append(avg_fit)
        history["best_ind"].append(best_ind)
        history["best_overall_fit"].append(best_overall_fit)

        # print per generație (best individual)
        if print_best_each_gen:
            print(
                f"Gen {g+1:02d} | best_val_acc={best_fit:.4f} | avg={avg_fit:.4f}\n"
                f"        best_ind={best_ind}\n"
                f"        best_overall_fit={best_overall_fit:.4f}"
            )

        # STOP: doar după min_generations
        if (g + 1) >= min_generations and best_fit >= fitness_target:
            print(f"[STOP] Prag atins în gen {g+1}: val_acc={best_fit:.4f}")
            break

        if stagnation >= patience:
            print(f"[STOP] Stagnare {patience} generații (gen {g+1}).")
            break

        # elitism
        elite_idxs = np.argsort(fits)[-elite:]
        new_pop = [dict(pop[i]) for i in elite_idxs]

        # reproducere
        while len(new_pop) < pop_size:
            p1 = tournament_selection(pop, fits, k=3)
            p2 = tournament_selection(pop, fits, k=3)

            child = crossover(p1, p2) if random.random() < cx_rate else dict(random.choice([p1, p2]))
            child = mutate(child, prob=mut_rate)
            new_pop.append(child)

        pop = new_pop

    # plot fitness (cu generațiile reale)
    gens = history["gen"]
    plt.figure()
    plt.plot(gens, history["best_fit"], marker="o", label="Best val_acc")
    plt.plot(gens, history["avg_fit"], marker="o", label="Avg val_acc")
    plt.xlabel("Generation")
    plt.ylabel("Validation accuracy")
    plt.title("GA: evoluția fitness-ului pe MNIST")
    plt.legend()
    plt.show()

    return best_overall, best_overall_fit, history


In [12]:
# ============================================================
# 6) Train FINAL model with best hyperparams + evaluate on test
# ============================================================

def train_final(best_ind, x_tr, y_tr, x_va, y_va, x_test, y_test, final_epochs=10, seed=42):
    tf.keras.utils.set_random_seed(seed)
    K.clear_session()

    model = build_cnn(best_ind)

    history = model.fit(
        x_tr, y_tr,
        validation_data=(x_va, y_va),
        epochs=final_epochs,
        batch_size=best_ind["batch_size"],
        verbose=1
    )

    test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
    print("\nFINAL TEST RESULTS")
    print("  test_loss =", float(test_loss))
    print("  test_acc  =", float(test_acc))

    return model, history

In [13]:
# ============================================================
# 7) RUN everything
# ============================================================

x_tr, y_tr, x_va, y_va, x_test, y_test = load_mnist(val_ratio=0.1, seed=7)

In [ ]:
best_ind, best_fit, history = run_ga(
    x_tr, y_tr, x_va, y_va,
    pop_size=10,
    generations=10,
    min_generations=4,
    fitness_target=0.99,
    patience=3,
    seed=7
)

print("\nBEST OVERALL:")
print(best_ind)
print("best_fit:", best_fit)

print("\nBEST PER GENERATION:")
for g, ind, fit in zip(history["gen"], history["best_ind"], history["best_fit"]):
    print(f"Gen {g:02d} | best_fit={fit:.4f} | {ind}")

print("\nBest hyperparameters found by GA:")
print(best_ind)
print("Best validation accuracy (fitness):", best_fit)

final_model, final_hist = train_final(
    best_ind,
    x_tr, y_tr, x_va, y_va, x_test, y_test,
    final_epochs=10
)

# plots
plt.figure()
plt.plot(final_hist.history["loss"], label="train_loss")
plt.plot(final_hist.history["val_loss"], label="val_loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Loss (model final)")
plt.legend()
plt.show()

plt.figure()
plt.plot(final_hist.history["accuracy"], label="train_acc")
plt.plot(final_hist.history["val_accuracy"], label="val_acc")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Accuracy (model final)")
plt.legend()
plt.show()
